In [49]:
%%capture
%pip install pandas matplotlib requests fastparquet

In [50]:
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

In [51]:
df = pd.read_parquet('data/trusted/df.parquet')

In [52]:
df = df[df['ct_classificacao'] == 'PF']

In [53]:
df.dtypes

data_base                            datetime64[ns]
ct_uf                                      category
ct_tcb                                     category
ct_sr                                      category
ct_cliente                                 category
ct_ocupacao                                category
ct_cnae_secao                              category
ct_cnae_subclasse                          category
ct_porte                                     object
ct_modalidade                                object
ct_origem                                  category
ct_indexador                               category
nu_numero_de_operacoes                        int64
vl_a_vencer_ate_90_dias                     float64
vl_a_vencer_de_91_ate_360_dias              float64
vl_a_vencer_de_361_ate_1080_dias            float64
vl_a_vencer_de_1081_ate_1800_dias           float64
vl_a_vencer_de_1801_ate_5400_dias           float64
vl_a_vencer_acima_de_5400_dias              float64
vl_vencido_a